In [14]:
from src.llm.llm_factory import LLMFactory

model = LLMFactory.create_llm()

In [27]:
# 讲笑话机器人： 希望每次根据指令， 可以输出一个这样的笑话（小明是怎么死的？笨死的）

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, field_validator
from typing import List
class Joke(BaseModel):
    setup: str = Field(description="设置笑话的问题")
    punchline: str = Field(description="回答笑话的答案")
    
    @field_validator("setup")
    def question_mark(cls, field):
        if not field.endswith("？"):
            raise ValueError("Setup must end with a question mark.")
        return field
# 将joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)
prompt = PromptTemplate(
    template = "回答用户的输入. \n{format_instrc}\n{query}\n",
    input_variales = ["query"],
    partial_variables = {"format_instrc": parser.get_format_instructions()}
)
pipline = prompt | model
output = pipline.invoke({"query": "给我讲一个笑话"})
print(output.content)
parser.parse(output.content)


```json
{
  "setup": "为什么电脑永远不会感冒？",
  "punchline": "因为它有Windows（窗户）！"
}
```


Joke(setup='为什么电脑永远不会感冒？', punchline='因为它有Windows（窗户）！')

In [31]:
# LLm的输出格式为Python List 形式 列入 ['a', 'b', 'c']

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instruction}",
    input_variables = ["subject"],
    partial_variables = {"format_instruction": parser.get_format_instructions() }
)
prompt.format(subject = "常见的中国人名字")
pipline = prompt | model
output = pipline.invoke({"subject": "常见的中国人名字"})

print(parser.parse(output.content))



['李明', '张伟', '王芳', '李娜', '刘强']
